In [1]:
import pickle
import sys
sys.path.append('../')
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pack_padded_sequence
from torch.nn.utils.rnn import pad_packed_sequence

from src.dataset import SquadDataset
from src.preprocessing import Preprocessing

# Clear memory
# torch.cuda.empty_cache()

# Notebook summary
In this notebook we'll set up the model architectures required for the first encoders. These encode the words in the documents, and the words in the questions. Both questions and documents are initially encoded by an LSTM:

$$ d_t = LSTM_{enc}(d_{t−1}, x_t^D) $$

resulting in document encoding matrix

$$ D = [d1, . . ., d_m, d_∅] \text{ of } L * (m+1) \text{ dimensions} $$

and 

$$ q_t = LSTM_{enc}(q_{t−1}, x_t^D) $$

resulting in intermediate question encoding matrix

$$ Q' = [q_1, . . ., q_n, q_∅] \text{ of } L * (n+1) \text{ dimensions} $$

to which we then apply a nonlinearity

$$ Q = tanh(W^{(Q)}Q_0 + b(Q)) \text{ of } L * (n+1) \text{ dimensions}

In [10]:
# Paths
glove_file_path = "../data/glove.840B.300d.txt"
squad_file_path = "../data/train-v1.1.json"

In [6]:
class DocumentEncoderLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(DocumentEncoderLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers)
    
    def forward(self, x):
        out = self.lstm(x)
        return out

### Define input data

Do we have a single document encoding matrix for all docuemnts, or an encoding matrix for each document, where L is the length of the transformed word vectors and m+1 is the number of words in the document plus a sentinel vector.

The shape of the input of a neural net is always defined on the level of a single example, as the batch size may vary. The above would suggest that we feed the network word vectors for a whole document. We pass each word vector through the same LSTM and we obtain new, encoded vectors (which incorporate some of their surrounding context).

This raises another question: how are we training this encoding? It seems we do not have a target to train on and therefore no error signal, at least in this section on its own. Just feeding the vectors through an LSTM with random weights seems a little pointless. It seems more likely that this is learned by going through the whole architecture. Does this mean that in order to test this we need to have the whole thing set up?

After we have both encodings D and Q, we calculate affinity matrix L = (D.transpose Q). This makes it unlikely that the encoders are coupled to the whole network, since it is difficult (impossible?) to disentangle the error signal you backpropagate.

SOLUTION: encoders are unsupervised, and they try to learn a mapping from x to x, e.g. they approximate the identity function. So we train the LSTM with backprop and pass our input along as targets. Conceptually, we have the word vectors, which encode meaning of single words. We pass these through an LSTM, which learns word context. So as output we get the same word meanings, which somehow also encapsulate word interactions because they have been through the LSTM. Is this correct??

In [4]:
# Set parameters
# Assuming that the LSTM takes one word at a time and the sizes stay the same through the encoder
input_size = 300
hidden_size = 300
output_size = 300
num_layers = 2
batch_size = 4
learning_rate = 0.0007
num_epochs = 10

In [7]:
# Setup model
model = DocumentEncoderLSTM(input_size, hidden_size, num_layers)
model.cuda()
lossfun = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

Because we're encoding the data we are learning the identity function. This means we use input data x as our target. This is a 3D Tensor, and the go-to loss function CrossEntropyLoss expects a 2D Tensor (usually labels are 1D, for every example, so 2D). Should we flatten our x? On the other hand, as it's not really classes we're predicting, it might be more intuitive to use the MSE or something similar.

In [11]:
# Get data
data = SquadDataset(squad_file_path, glove_file_path, target='text')

Found pickled GloVe file. Loading...
Done. 2195875 words loaded!


In [12]:
dataloader = DataLoader(data, batch_size=batch_size, shuffle=True)

In [ ]:
for epoch in range(num_epochs):
    for i, data_batch in enumerate(dataloader):
        x = Variable(data_batch['text'].float())
        x = x.cuda()
        y = x
        
        output = model(x)
        optimizer.zero_grad()
        loss = lossfun(output[0],y)
        loss.backward()
        optimizer.step()
        
        if (i+1)%100 ==0:
            print('Epoch [%d/%d], Step[%d/%d], Loss: %0.4f'
                 %(epoch+1, num_epochs, i+1, len(data)//batch_size, loss.data[0]))